### 1. 데이터 생성하기
(1) ./datasets 아래에 .hwp 파일을 모두 load하기

(2) text_splitter로 분할하기

(3) all_docs에 모든 chunk들 append하기

(4) Pineone Vector database에 업로드하기

In [29]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_teddynote.document_loaders.hwp import HWPLoader
import os
import re

document_list = []

# text splitter 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=100,
)

# 불용어 리스트
stopwords = ['은', '는', '이', '가', '을', '를', '에', '의', '도', '에서', '으로']

def preprocess_text(text):
    text = re.sub(r'[^\w\s가-힣]', '', text)  # 한글+영어 외 특수문자 제거
    text = re.sub(r'\s+', ' ', text).strip()  # 중복 공백 제거
    text = ' '.join([word for word in text.split() if word not in stopwords])  # 불용어 제거
    return text

# datasets 폴더를 시작점으로 재귀 순회
for root, dirs, files in os.walk("./datasets"):
    for file in files:
        if file.endswith(".hwp"):
            file_path = os.path.join(root, file)
            loader = HWPLoader(file_path)
            docs = loader.load()
            for i, doc in enumerate(docs):
                doc.page_content = preprocess_text(doc.page_content)
                split_docs = text_splitter.split_documents([doc])
                #메타 데이터 추가
                for idx, split_doc in enumerate(split_docs):
                    split_doc.metadata = {
                        'file_name': file,
                        'file_path': file_path,
                        'chunk_id': idx
                    }
                document_list.extend(split_docs)
                
# 중복 청크 제거
document_list = list({doc.page_content: doc for doc in document_list}.values())

# 너무 짧은 청크 제거
document_list = [doc for doc in document_list if len(doc.page_content) > 20]

print(f"The number of unique and filtered chunks: {len(document_list)}")

The number of unique and filtered chunks: 2702


In [35]:
document_list[6].metadata['file_name']

'조교인사규정_20171108(20240423)_일부개정.hwp'

In [30]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [36]:
#Pinecone
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'ds-markdown-clean'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [11]:
#Chroma
# from langchain_chroma import Chroma

# database=Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma-tax', persist_directory="./chroma")

#database=Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embedding) #이미 만들어 놓은 데베 사용

### 2. 답변 생성을 위한 Retrieval 및 Prompt 활용하기
(1) RetrievalQA에 전달하기 위해 retriever 생성

(2) LangChain에서 제공하는 rlm/rag-prompt 사용

In [37]:
query = '컴퓨터공학전공은 심화 이수하기 위해서는 졸업조건이 무엇인가요?'
retriever = database.as_retriever(search_kwargs={'k': 4})
retriever.invoke(query)

[Document(id='16edfc02-3fea-4780-8f99-ed1d46385952', metadata={'chunk_id': 7.0, 'file_name': '외국대학과의 복수학위제 운영 규정(20240403)_일부개정.hwp', 'file_path': './datasets/3편 학사행정/4장 교학행정/외국대학과의 복수학위제 운영 규정(20240403)_일부개정.hwp'}, page_content='규정은 2022년 7월 1일부터 시행한다부 칙2024 4 31 시행일 개정 규정은 2024년 4월 3일부터 시행한다별표 1 복수학위생 최대 학점인정 범위 신설 2021 2 24 개정 2024431 본교 복수학위생의 최대 학점인정 범위22 입학년도이수구분 20172018학년도2019학년도전공복수전공22학점심화전공31학점부전공15학점연계전공18학점총 학점65학점 입학년도이수구분 2020학년도2021학년도 이후전공제1전공 제2전공18학점심화전공31학점총 학점65학점2 본교 복수학위생의 최대 학점인정 범위31 입학년도이수구분 20172018학년도2019학년도전공복수전공33학점심화전공47학점부전공22학점연계전공27학점총 학점97학점 입학년도이수구분 2020학년도2021학년도 이후전공제1전공 제2전공27학점디지털소프트웨어공학부 33학점심화전공47학점총 학점97학점3 외국대학 복수학위생의 최대 학점인정 범위22 입학년도이수구분 20172018학년도2019학년도교양32학점34학점전공복수전공23학점심화전공31학점총'),
 Document(id='52df48b6-007f-47bc-8c33-6c5ea655fa60', metadata={'chunk_id': 3.0, 'file_name': '전공모듈형과정운영규정(20250124)_신규제정.hwp', 'file_path': './datasets/3편 ᄒ

In [38]:
query = '본교에서 컴퓨터공학전공은 심화 이수하기 위해서는 졸업조건이 무엇인가요?'
retriever = database.as_retriever(search_kwargs={'k': 4})
retriever.invoke(query)

[Document(id='ff6602df-5ceb-405f-9091-008ec2793afe', metadata={'chunk_id': 59.0, 'file_name': '덕성여자대학교학칙(20240625)_일부개정.hwp', 'file_path': './datasets/2편 학칙/덕성여자대학교학칙(20240625)_일부개정.hwp'}, page_content='그 소속을 공과대학 IT미디어공학과로 변경한다5 컴퓨터학과 명칭 및 소속 변경에 따른 경과조치 개정 학칙 시행일 전 입학한 정보미디어대학 컴퓨터학과 재적생 및 2018학년도 공과대학 컴퓨터학과 입학생은 2019년 3월 1일자로 그 소속을 공과대학 컴퓨터공학과로 변경한다부 칙20184231 시행일 개정 학칙은 2018년 4월 23일부터 시행한다2 교육조직 학위수여 적용례 제3조 개정 규정의 공과대학은 2018학년도부터 적용하며 정보미디어대학은 단과대학내 컴퓨터학과의 재적생이 있는 기간2018학년도까지 존속하고 해당 기간 컴퓨터학과의 학위는 개정 전 학칙별표 2에 의거 수여한다3 학부 학과 입학정원 적용례 제4조 제1항 별표 1의 개정 사항은 2019학년도 입학생부터 적용한다 부 칙20191241 시행일 개정 학칙은 2019년 1월 24일부터 시행한다 다만 제52조의 개정 규정은 2019년 3월 1일부터 시행한다2 재학연한에 관한 적용례 제5조제2항부터 제6항까지의 개정 규정은 2014학년도 입학자부터'),
 Document(id='16edfc02-3fea-4780-8f99-ed1d46385952', metadata={'chunk_id': 7.0, 'file_name': '외국대학과의 복수학위제 운영 규정(20240403)_일부개정.hwp', 'file_path': './datasets/3편 학사행정/4장 교학행정/외국대학과의 보

In [39]:
from langchain import hub
from langchain_openai import ChatOpenAI

prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model='gpt-4o')

### 3. 답변 생성하기
(1) RetrievalQA를 통해 LLM에 전달

In [40]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

ai_message = qa_chain.invoke({"query": query})

ai_message

{'query': '본교에서 컴퓨터공학전공은 심화 이수하기 위해서는 졸업조건이 무엇인가요?',
 'result': '본교에서 컴퓨터공학전공을 심화 이수하기 위해서는 47학점을 이수해야 합니다.'}